In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn import svm
import pandas as pd
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from ast import literal_eval
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, confusion_matrix, ConfusionMatrixDisplay
from sklearn import svm
from sklearn.model_selection import RandomizedSearchCV, train_test_split
import numpy as np
files=glob.glob('pre_processed_training/*.gzip')
total_training_data=pd.DataFrame([])
for f in tqdm(files):
    total_training_data=pd.concat([total_training_data,pd.read_parquet(f, engine='auto')],axis=0)
total_training_data=total_training_data.sort_values(['Subject','Session','time_stamp'])
total_training_data=total_training_data.reset_index()
total_training_data['epoch'] = total_training_data['time_stamp'].sub(pd.Timestamp('1970-01-01 00:00:00.000'))
total_training_data['epoch']=total_training_data['epoch'].dt.total_seconds()
total_training_data=total_training_data[['Subject','Session','epoch','measurements','labels']]
total_training_data_grouped=total_training_data.groupby(['Subject','Session','epoch']).agg({'measurements':[np.mean],'labels':[np.mean]})
total_training_data_grouped.columns = total_training_data_grouped.columns.droplevel(1)
total_training_data_grouped=total_training_data_grouped.reset_index(level=['epoch'])
total_training_data_grouped.to_parquet(f"combined_sampled/training.gzip",compression='gzip')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 33.60it/s]


In [15]:
total_training_data_g=total_training_data_g.reset_index(level=['epoch'])

In [16]:
total_training_data_g

epoch                                       measurements  \
Subject Session                                                                
1       1           0.000  [4.435275, 8.196063, 2.974488, 0.01421523, -0....   
        1           0.025  [4.18692, 8.344455, 2.908057, 0.005770759, -0....   
        1           0.050  [4.544637, 8.408659, 2.89, 0.007967429, 0.0224...   
        1           0.075  [4.849308, 8.411614, 2.900692, 0.02777778, -0....   
        1           0.100  [4.50919, 8.118649, 2.847298, 0.02157696, -0.0...   
...                   ...                                                ...   
8       1        1203.325  [2.072244, 8.908878, -3.5, 0.001350761, 0.0011...   
        1        1203.350  [2.085123, 8.915123, -3.52, 0.001917847, -0.00...   
        1        1203.375  [2.083774, 8.91, -3.538981, -0.002015218, -0.0...   
        1        1203.400  [2.111447, 8.908553, -3.535724, 0.0001831479, ...   
        1        1203.425  [2.08773, 8.9, -3.518865, 5.186139e-05, -0.001...   

                 labels  
Subject Session          
1       1           0.0  
        1           0.0  
        1           0.0  
        1           0.0  
        1           0.0  
...                 ...  
8       1           0.0  
        1           0.0  
        1           0.0  
        1           0.0  
        1           0.0  

[1341646 rows x 3 columns]

In [ ]:
total_training_data_grouped

In [ ]:
total_training_data_g=total_training_data_grouped.copy()

In [ ]:
total_training_data_g.columns = total_training_data_g.columns.droplevel(1)

In [ ]:
total_training_data_g

In [ ]:
total_training_data['time_stamp'][2].time()

In [ ]:
s_time_interv=total_training_data.groupby(pd.Grouper(freq=))

In [ ]:
pr=list(total_training_data.iloc[4])

In [ ]:
total_training_data.iloc[4].to_dict()

In [ ]:
pr.index

In [ ]:
subject=1
sample=total_training_data

s_time_interv= total_training_data[['Subject','Session','time_stamp']]
s_time_interv=s_time_interv.groupby(['Subject','Session']).agg({'time_stamp': [np.min,np.max]})
s_time_interv.columns=s_time_interv.columns.droplevel()
s_time_interv['time_interval']=s_time_interv['amax']-s_time_interv['amin']
s_time_interv['time_interval']=s_time_interv['time_interval'].dt.total_seconds()
s_time_interv=s_time_interv[['time_interval']]
s_time_interv=s_time_interv.unstack()
s_time_interv.columns=s_time_interv.columns.droplevel()
#s=s.reset_index(drop=True)

In [ ]:
s_time_interv.plot(kind='bar',ylabel='time_interval (s)',figsize=(8,4),title=f"Subject-Session-Interval Distribution")
plt.xticks(rotation='horizontal')
SMALL_SIZE = 12
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5)).set_title("Session")
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title

In [ ]:
SMALL_SIZE = 8
matplotlib.rc('font', size=SMALL_SIZE)
matplotlib.rc('axes', titlesize=SMALL_SIZE)

In [ ]:

#s_label_dist.columns=s_label_dist.columns.droplevel()
ax=s_label_dist.plot(kind='bar',xlabel='labels',ylabel='count',legend=False)
ax.bar_label(ax.containers[0])

In [ ]:
ax = df.plot.bar()



In [ ]:
sample=total_training_data[(total_training_data.Subject==8)].reset_index()

In [ ]:
s_label_dist= sample[['labels','time_stamp']]
s_label_dist=s_label_dist.groupby(['labels']).count().rename(columns={'time_stamp':'Count','labels':'Labels'}).reset_index()
s_label_dist['Labels']=['Solid Ground(0)','Downstairs(1)','Upstairs(2)','Grass(3)']
ax=sns.barplot(x = 'Labels',y = 'Count',data = s_label_dist)
ax.bar_label(ax.containers[0])
plt.show()

In [ ]:
s_label_dist= total_training_data[['Session','labels','time_stamp']]
s_label_dist=s_label_dist.groupby(['Session','labels']).count()
s_label_dist=s_label_dist.unstack()
s_label_dist.columns=s_label_dist.columns.droplevel()
s_label_dist.plot(kind='bar',xlabel='Session',ylabel='count')

In [ ]:
s1_s1=total_training_data[(total_training_data.Subject==1) & (total_training_data.Session==1)][['time_stamp','measurements','labels']]

In [ ]:
s1_s1_grouped=s1_s1.set_index('time_stamp').groupby([pd.Grouper(freq='1min'),'labels']).count().unstack()
s1_s1_grouped.columns=s1_s1_grouped.columns.droplevel()
s1_s1_grouped.index=s1_s1_grouped.index.time
s1_s1_grouped.plot()

In [ ]:
s1_s1

In [ ]:
s1_s1['ranks']=s1_s1.groupby(['labels'])['time_stamp'].rank(method='average')

In [ ]:
s1_s1.groupby(['ranks','labels']).agg({'time_stamp': [np.min,np.max]})

In [ ]:
s1_s1['ranks']=s1_s1['labels'].diff(1)

In [ ]:
s1_s1[s1_s1.labels==1]

In [ ]:
s1_s1['time_stamp']=s1_s1['time_stamp']-s1_s1['time_stamp'][0]
s1_s1['time_stamp']=s1_s1['time_stamp'].dt.total_seconds()

In [ ]:
s1_s1

In [ ]:
derive = (s1_s1.labels != s1_s1.labels.shift()).cumsum()
#Group records by user, location and the calculated column, and then sum duration values
res = s1_s1.groupby(['labels', derive], as_index=False, sort=False)[['time_stamp']].mean()
res['labels']=res['labels'].apply(lambda x: str(x))
# res=res.set_index('time_stamp')
# res.index=res.index.time
# res=res.reset_index().rename(columns={'index':'time_stamp'})
#res.columns=res.columns.droplevel()
#res['time_interval']=res['amax']-res['amin']
#res['time_interval']=res['time_interval'].dt.total_seconds()


In [ ]:
res

In [ ]:
res=res.pivot(columns='labels')
res.columns=res.columns.droplevel()

In [ ]:
res

In [ ]:
total_training_data

In [ ]:
!pip install yellowbrick